In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

import seaborn as sns

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/train.csv")
train_df.head()

In [ ]:
train_df.shape

In [ ]:
ax = sns.countplot(x="claim", data=train_df)

In [ ]:
def fillup_missing_values(df, col):
  column_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
  df[col] = column_imputer.fit_transform(df.loc[:,[col]])

In [ ]:
#for x in train_df.columns:
#    if train_df[x].isnull().sum() > 0:
#       fillup_missing_values(train_df, x)

In [ ]:
#train_df.dropna(inplace=True)

In [ ]:
y = train_df.pop('claim')

In [ ]:
features_to_drop = ['id']

In [ ]:
X = train_df.drop(columns=features_to_drop)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 0, stratify=y)
#X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 0)

In [ ]:
#### model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=200, scale_pos_weight=2, learning_rate=0.3, gamma=0.5, reg_lambda=10, subsample=0.5)
model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=1000, learning_rate=0.1,subsample=0.8,colsample_bytree=0.5, reg_lambda=10, max_depth=5)
model.fit(X_train, y_train, verbose=True, early_stopping_rounds=20, eval_metric='auc', eval_set=[(X_test, y_test)]),

In [ ]:
plot_confusion_matrix(model, X_test, y_test)

In [ ]:
#param_grid = {
#    'max_depth':[3,4,5],
#    'learning_rate':[0.3, 0.5, 0.7],
#    'gamma':[0, 0.25, 1],
#    'reg_lambda':[1,5,10],
#    'scale_post_weight':[1,3,5]
#}

#optimal_params = GridSearchCV(estimator=xgb.XGBClassifier(objective='binary:logistic', subsample=0.9,colsample_bytree=0.5),
#                             param_grid=param_grid,scoring='roc_auc',verbose=0,n_jobs=10,cv=3)

#optimal_params.fit(X_train, y_train, early_stopping_rounds=10, eval_metric='auc',eval_set=[(X_test, y_test)],verbose=False)

#print(optimal_params.best_params_)

In [ ]:
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/test.csv")
test_df

In [ ]:
#for x in test_df.columns:
#    if test_df[x].isnull().sum() > 0:
#        fillup_missing_values(test_df, x)

In [ ]:
#test_df.dropna(inplace=True)

In [ ]:
X_test = test_df.drop(columns=features_to_drop)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
output = pd.DataFrame({'id': test_df.id, 'claim': y_pred})
print(output.head(20))
output.to_csv('xgboost_submission.csv', index=False)